In [ ]:
import pandas as pd
import wfdb
import os
import numpy as np
from scipy.signal import butter, filtfilt, find_peaks, convolve
import matplotlib.pyplot as plt

In [ ]:
root_path = r"C:\Users\SGUZEY\Documents\Yüksek Lisans Dersleri\İleri Gömülü Sistemler\Data Sets\mit-bih-arrhythmia-database-1.0.0"

orijinal_records = []

for root, dirs, files in os.walk(root_path):
    for f in files:
        if f.endswith('.hea'):
            rec_name = os.path.splitext(f)[0]

            if not rec_name.startswith('x_'):
                full_path = os.path.join(root, f)
                orijinal_records.append((rec_name, root))

orijinal_records = sorted(list(set(orijinal_records)))

In [ ]:
all_data = []

for rec, rec_path in orijinal_records:
    try:
        record = wfdb.rdrecord(os.path.join(rec_path, rec))
        annotation = wfdb.rdann(os.path.join(rec_path, rec), 'atr')

        df = pd.DataFrame(record.p_signal, columns=record.sig_name)
        df['record'] = rec
        df['annotation'] = None

        for i, idx in enumerate(annotation.sample):
            if idx < len(df):
                df.loc[idx, 'annotation'] = annotation.symbol[i]

        all_data.append(df)

    except Exception as e:
        print(f"{rec} not read: {e}")

merged_df = pd.concat(all_data, ignore_index=True)

In [ ]:
if all_data:
    merged_df = pd.concat(all_data, ignore_index=True)
    print(f"Total of sample's number: {len(merged_df):,}")
merged_df.head()

In [ ]:
first = merged_df.iloc[:650000].copy()
Fs = 360
ekg_signal_raw = first['MLII'].values

N = len(ekg_signal_raw)
signal_time = np.arange(N) / Fs

In [ ]:
f_low_pass_cut = 15.0
f_high_pass_cut = 5.0

def calculate_filter_coeffs(cutoff_freq, filter_type, Fs=360):
    Nyquist = Fs / 2
    order = 4 # 4. dereceden Butterworth filtresi
    normalized_cutoff = cutoff_freq / Nyquist
    b, a = butter(order, normalized_cutoff, btype=filter_type, analog=False)
    return b, a

def pan_tompkins_filter(ekg_signal):
    # Düşük Geçiren Filtre (Low Pass Filter - LPF)
    b_lpf, a_lpf = calculate_filter_coeffs(f_low_pass_cut, 'low')
    ekg_lpf = filtfilt(b_lpf, a_lpf, ekg_signal)

    # Yüksek Geçiren Filtre (High Pass Filter - HPF)
    b_hpf, a_hpf = calculate_filter_coeffs(f_high_pass_cut, 'high')
    ekg_bpf = filtfilt(b_hpf, a_hpf, ekg_lpf)

    return ekg_bpf

ekg_signal_filtered = pan_tompkins_filter(ekg_signal_raw)

In [ ]:
deriv_coeffs = np.array([-1, -2, 0, 2, 1]) / 8.0 * Fs
ekg_signal_differentiation = convolve(ekg_signal_filtered, deriv_coeffs, mode='same')

In [ ]:
ekg_kareli = ekg_signal_differentiation**2

In [ ]:
MWI_WINDOW_MS = 150
MWI_WINDOW_SAMPLES = int(MWI_WINDOW_MS * Fs / 1000)

if MWI_WINDOW_SAMPLES < 1:
    MWI_WINDOW_SAMPLES = 1

mwi_kernel = np.ones(MWI_WINDOW_SAMPLES) / MWI_WINDOW_SAMPLES
ekg_mwi = convolve(ekg_kareli, mwi_kernel, mode='same')

In [ ]:
INIT_WINDOW_SIZE = int(0.5 * Fs)
QRS_WINDOW_MS = 200
QRS_WINDOW_SAMPLES = int(QRS_WINDOW_MS * Fs / 1000)
T_WAVE_WINDOW_SAMPLES = int(360 * Fs / 1000)

max_initial_peak = np.max(ekg_mwi[:INIT_WINDOW_SIZE])
SPKI = max_initial_peak * 0.9
NPKI = np.mean(ekg_mwi[:INIT_WINDOW_SIZE]) * 0.5

QRS_LOCS = []
LAST_QRS_INDEX = 0

N_total = len(ekg_mwi)
SPKI_history = np.zeros(N_total)
NPKI_history = np.zeros(N_total)
TH1_history = np.zeros(N_total)

peaks, _ = find_peaks(ekg_mwi, height=0, distance=QRS_WINDOW_SAMPLES // 2)

for i in range(N_total):
    peak_val = ekg_mwi[i]

    TH1 = NPKI + 0.25 * (SPKI - NPKI)
    TH2 = 0.5 * TH1

    SPKI_history[i] = SPKI
    NPKI_history[i] = NPKI
    TH1_history[i] = TH1

    if i in peaks:
        is_refractory = (i - LAST_QRS_INDEX) < QRS_WINDOW_SAMPLES
        is_t_wave_range = (i - LAST_QRS_INDEX) < T_WAVE_WINDOW_SAMPLES

        if peak_val > TH1:
            if not is_refractory:
                QRS_LOCS.append(i)
                LAST_QRS_INDEX = i

                SPKI = 0.125 * peak_val + 0.875 * SPKI
            else:

                SPKI = 0.125 * peak_val + 0.875 * SPKI

        elif peak_val > TH2:
            if not is_t_wave_range and not is_refractory:

                QRS_LOCS.append(i)
                LAST_QRS_INDEX = i
                SPKI = 0.25 * peak_val + 0.75 * SPKI

            elif is_t_wave_range:
                NPKI = 0.125 * peak_val + 0.875 * NPKI

        else:
            NPKI = 0.125 * peak_val + 0.875 * NPKI

QRS_LOCS = np.array(QRS_LOCS)
QRS_LOCS_TIME = QRS_LOCS / Fs

In [ ]:
RR_INTERVALS = np.diff(QRS_LOCS_TIME)

if len(RR_INTERVALS) > 0:
    QRS_LOCS_MEAN = np.mean(RR_INTERVALS)
    QRS_LOCS_STD = np.std(RR_INTERVALS)
    AVERAGE_HR_BPM = 60 / QRS_LOCS_MEAN
else:
    QRS_LOCS_MEAN = 0
    QRS_LOCS_STD = 0
    AVERAGE_HR_BPM = 0

print(f"Tespit Edilen QRS Sayısı: {len(QRS_LOCS)}")
print(f"Toplam Süre (saniye): {len(ekg_signal_raw) / Fs:.2f}")

print("\n--- RR Aralık İstatistikleri ---")
print(f"Ortalama RR Aralığı (saniye): {QRS_LOCS_MEAN:.4f} s")
print(f"RR Aralığı Standart Sapması (saniye): {QRS_LOCS_STD:.4f} s")

print("\n--- Kalp Atış Hızı Tahmini ---")
print(f"Ortalama Kalp Atış Hızı (BPM): {AVERAGE_HR_BPM:.2f} atım/dakika")

In [ ]:
target_time = 10
end_index = int(Fs * target_time)
time_to_plot = signal_time[:end_index]

signal_to_plot_raw = ekg_signal_raw[:end_index]
signal_to_plot_filtered = ekg_signal_filtered[:end_index]
signal_deriv_to_plot = ekg_signal_differentiation[:end_index]
signal_squared_to_plot = ekg_kareli[:end_index]
signal_mwi_to_plot = ekg_mwi[:end_index]

SPKI_to_plot = SPKI_history[:end_index]
NPKI_to_plot = NPKI_history[:end_index]
TH1_to_plot = TH1_history[:end_index]

qrs_in_plot_range_indices = [idx for idx in QRS_LOCS if idx < end_index]
qrs_in_plot_range_times = np.array(qrs_in_plot_range_indices) / Fs
qrs_in_plot_range_vals = ekg_mwi[qrs_in_plot_range_indices]

# --- 1. FIGÜR GRUBU (Ham, Filtrelenmiş ve Türevlenmiş Sinyaller) ---
# AMAÇ: Her adımı ayrı bir figürde gösterip kaydetmek.

# 1. Ham EKG Sinyali
plt.figure(figsize=(15, 5))
plt.plot(time_to_plot, signal_to_plot_raw, 'b-', label='Ham EKG (MLII)')
plt.title(f'1. Ham ECG Sinyali - İlk {target_time} Saniye')
plt.ylabel('Amplitüd')
plt.xlabel('Zaman (saniye)') # X ekseni etiketi eklendi
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Filtreleme (Bant Geçiren Filtre)
plt.figure(figsize=(15, 5))
plt.plot(time_to_plot, signal_to_plot_filtered, color='purple', label='Bant Geçiren Filtre Çıktısı')
plt.title('2. Filtreleme (5-15 Hz Bandı)')
plt.ylabel('Amplitüd')
plt.xlabel('Zaman (saniye)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 3. Türev Alma
plt.figure(figsize=(15, 5))
plt.plot(time_to_plot, signal_deriv_to_plot, color='orange', label='Türevlenmiş Sinyal')
plt.title('3. Türev Alma')
plt.xlabel('Zaman (saniye)')
plt.ylabel('Türev Amplitüdü')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# --- 2. FIGÜR GRUBU (Kare Alma, MWI ve QRS Tespiti) ---

# 4. Kare Alma Çıktısı
plt.figure(figsize=(15, 5))
plt.plot(time_to_plot, signal_squared_to_plot, 'g-', label='Karesi Alınmış Sinyal (Türev^2)')
plt.title('4. Kare Alma Çıktısı')
plt.ylabel('Kare Amplitüdü')
plt.xlabel('Zaman (saniye)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 5. Hareketli Pencere Bütünlemesi (MWI) Çıktısı
plt.figure(figsize=(15, 5))
plt.plot(time_to_plot, signal_mwi_to_plot, color='red', label='MWI Çıktısı (QRS Enerjisi)')
plt.title(f'5. Hareketli Pencere Bütünlemesi Çıktısı (Pencere: {MWI_WINDOW_MS}ms)')
plt.ylabel('Enerji/Amplitüd')
plt.xlabel('Zaman (saniye)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 6. Adaptif Eşikleme ile QRS Tespiti (Bu grafiği tek bir figürde tutmak daha mantıklı)
plt.figure(figsize=(15, 5))
plt.plot(time_to_plot, signal_mwi_to_plot, 'b-', label='MWI Sinyali')
plt.plot(time_to_plot, NPKI_to_plot, 'k--', label='Gürültü Seviyesi (NPKI)')
plt.plot(time_to_plot, SPKI_to_plot, 'r--', label='Sinyal Seviyesi (SPKI)')
plt.plot(time_to_plot, TH1_to_plot, 'g--', label='Adaptif Eşik (TH1)')

plt.scatter(qrs_in_plot_range_times, qrs_in_plot_range_vals,
            color='magenta', marker='*', s=80, zorder=5, label='Tespit Edilen QRS')

plt.title('6. Adaptif Eşikleme ile QRS Tespiti')
plt.xlabel('Zaman (saniye)')
plt.ylabel('Amplitüd (MWI Çıktısı)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- 3. FIGÜR GRUBU (HRV Analizi) ---

# 7. RR Aralıklarının Dağılımı
plt.figure(figsize=(10, 5))
plt.hist(RR_INTERVALS, bins=20, color='skyblue', edgecolor='black')
plt.axvline(QRS_LOCS_MEAN, color='red', linestyle='dashed', linewidth=2, label=f'Ortalama RR ({QRS_LOCS_MEAN:.4f} s)')
plt.title(f'RR Aralıklarının Dağılımı (Ortalama Kalp Hızı: {AVERAGE_HR_BPM:.2f} BPM)')
plt.xlabel('RR Aralığı (saniye)')
plt.ylabel('Sayı')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
STD_DEV_FACTOR = 2.0
lower_bound = QRS_LOCS_MEAN - STD_DEV_FACTOR * QRS_LOCS_STD
upper_bound = QRS_LOCS_MEAN + STD_DEV_FACTOR * QRS_LOCS_STD

outlier_rr_indices_qrs = np.where((RR_INTERVALS < lower_bound) | (RR_INTERVALS > upper_bound))[0] + 1

outlier_qrs_locs = QRS_LOCS[outlier_rr_indices_qrs]

regular_qrs_locs = np.array([loc for loc in QRS_LOCS if loc not in outlier_qrs_locs])

print(f"Toplam Tespit Edilen QRS Sayısı: {len(QRS_LOCS)}")
print(f"Ortalama RR Aralığı: {QRS_LOCS_MEAN:.4f} s")
print(f"Aykırı Alt Sınır: {lower_bound:.4f} s, Aykırı Üst Sınır: {upper_bound:.4f} s")
print(f"Aykırı (Outlier) QRS Sayısı: {len(outlier_qrs_locs)}")

WINDOW_SEC = 10
HALF_WINDOW_SAMPLES = int(WINDOW_SEC * Fs / 2)
MAX_PLOTS = 5

if len(outlier_qrs_locs) == 0:
    print("\nTespit edilen aykırı QRS noktası bulunmamaktadır. Görselleştirme atlandı.")
else:
    outliers_to_plot = outlier_qrs_locs[:MAX_PLOTS]
    num_plots = len(outliers_to_plot)

    print(f"\nTespit edilen aykırı QRS noktalarından ilk {num_plots} tanesi için 10 saniyelik grafikler oluşturuluyor...")

    regular_qrs_times = regular_qrs_locs / Fs
    regular_qrs_vals = ekg_mwi[regular_qrs_locs]

    outlier_qrs_times = outlier_qrs_locs / Fs
    outlier_qrs_vals = ekg_mwi[outlier_qrs_locs]

    for i, outlier_loc in enumerate(outliers_to_plot):
        start_index = max(0, outlier_loc - HALF_WINDOW_SAMPLES)
        end_index = min(len(ekg_mwi), outlier_loc + HALF_WINDOW_SAMPLES)

        window_signal = ekg_mwi[start_index:end_index]
        window_time = np.arange(start_index, end_index) / Fs

        center_time = outlier_loc / Fs
        relative_time = window_time - center_time + (WINDOW_SEC / 2)

        window_regular_qrs_locs = regular_qrs_locs[(regular_qrs_locs >= start_index) & (regular_qrs_locs < end_index)]
        window_outlier_qrs_locs = outlier_qrs_locs[(outlier_qrs_locs >= start_index) & (outlier_qrs_locs < end_index)]

        window_regular_qrs_times = window_regular_qrs_locs / Fs - center_time + (WINDOW_SEC / 2)
        window_regular_qrs_vals = ekg_mwi[window_regular_qrs_locs]

        window_outlier_qrs_times = window_outlier_qrs_locs / Fs - center_time + (WINDOW_SEC / 2)
        window_outlier_qrs_vals = ekg_mwi[window_outlier_qrs_locs]

        plt.figure(figsize=(15, 6))

        plt.plot(relative_time, window_signal, 'b-', label='MWI Sinyali')

        plt.scatter(window_regular_qrs_times, window_regular_qrs_vals,
                    color='green', marker='o', s=50, zorder=5, label='Düzenli QRS')

        plt.scatter(window_outlier_qrs_times, window_outlier_qrs_vals,
                    color='red', marker='X', s=100, zorder=6, label='Aykırı QRS (Outlier)')

        plt.axvline(x=WINDOW_SEC / 2, color='red', linestyle=':', alpha=0.6, label='Aykırı Merkez Nokta (5s)')

        plt.title(f'Aykırı QRS Tespiti ({i+1}/{num_plots}) - Merkez Zaman: {center_time:.2f} s')
        plt.xlabel(f'Göreceli Zaman (saniye) - Merkez Nokta t={WINDOW_SEC/2:.0f}s')
        plt.ylabel('Amplitüd (MWI Çıktısı)')
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
# Fs'i sabit olarak tanımlıyoruz (MIT-BIH için 360 Hz)
FS = 360
f_low_pass_cut = 15.0
f_high_pass_cut = 5.0
deriv_coeffs = np.array([-1, -2, 0, 2, 1]) / 8.0 * FS
MWI_WINDOW_MS = 150
MWI_WINDOW_SAMPLES = int(MWI_WINDOW_MS * FS / 1000)
if MWI_WINDOW_SAMPLES < 1:
    MWI_WINDOW_SAMPLES = 1
mwi_kernel = np.ones(MWI_WINDOW_SAMPLES) / MWI_WINDOW_SAMPLES

# Filtre katsayılarını hesaplayan yardımcı fonksiyon
def calculate_filter_coeffs(cutoff_freq, filter_type, Fs=FS):
    Nyquist = Fs / 2
    order = 4
    normalized_cutoff = cutoff_freq / Nyquist
    b, a = butter(order, normalized_cutoff, btype=filter_type, analog=False)
    return b, a

# Pan-Tompkins Sinyal İşleme Zinciri
def pan_tompkins_processor(ekg_signal):
    # Bant Geçiren Filtre
    b_lpf, a_lpf = calculate_filter_coeffs(f_low_pass_cut, 'low')
    ekg_lpf = filtfilt(b_lpf, a_lpf, ekg_signal)
    b_hpf, a_hpf = calculate_filter_coeffs(f_high_pass_cut, 'high')
    ekg_bpf = filtfilt(b_hpf, a_hpf, ekg_lpf)

    # Türevleme
    ekg_deriv = convolve(ekg_bpf, deriv_coeffs, mode='same')

    # Kare Alma
    ekg_squared = ekg_deriv**2

    # MWI
    ekg_mwi = convolve(ekg_squared, mwi_kernel, mode='same')

    return ekg_mwi

# Basit adaptif eşikleme ile R-Tepe Tespiti (Pan-Tompkins'in son adımı)
def detect_r_peaks(ekg_mwi):
    # MWI çıktısında pik tespiti
    # 'height' = Ortalama MWI değerinin belli bir katı (gürültü filtreleme)
    # 'distance' = Refrakter periyot (yaklaşık 200ms)
    distance_samples = int(0.2 * FS)

    # Çok basit eşik kullanıyoruz. Orijinal adaptif eşik daha kararlıdır ancak temel tespitte bu yeterli olabilir.
    # Burada yalnızca find_peaks'in kendi eşikleme mekanizmasını kullanıyoruz:
    peaks, _ = find_peaks(ekg_mwi, height=np.max(ekg_mwi) * 0.2, distance=distance_samples)

    # MWI pikleri, gerçek QRS (MLII) R-tepe pozisyonuna karşılık gelmelidir.
    # MWI'daki pik pozisyonları, filtrelenmiş MLII sinyalinde R-tepesini bulmak için kullanılır.

    # MLII sinyalindeki nihai R-tepe pozisyonlarını döndür
    return peaks

In [ ]:
def create_feature_df(orijinal_records, root_path):
    all_features = []

    # Tüm annotation sembollerini kategorilere eşlemek için bir harita (basitleştirilmiş)
    # Ref: AAMI Standardı ve WFDB dokümantasyonu
    # N: Normal, L: Left Bundle Branch Block, R: Right Bundle Branch Block, B: Bundle branch block beat
    # A: Atrial Premature, V: Ventricular Premature Contraction, F: Fusion, Q: Unknown
    annotation_map = {
        'N': 'Normal', 'L': 'Normal', 'R': 'Normal', 'e': 'Normal', 'j': 'Normal',
        'A': 'Supraventricular', 'a': 'Supraventricular', 'J': 'Supraventricular', 'S': 'Supraventricular',
        'V': 'Ventricular', 'E': 'Ventricular',
        'F': 'Fusion',
        '/': 'Paced', 'f': 'Fusion', 'Q': 'Unknown', '[': 'Start', '!': 'Unknown',
        ']': 'End', 'x': 'Unknown', '(': 'Other', ')': 'Other', 'p': 'Other', 't': 'Other',
        'u': 'Other', '`': 'Other', "'": 'Other', '^': 'Other', '|': 'Other', '~': 'Other',
        '+': 'Other', 's': 'Supraventricular', 'D': 'Other', '=': 'Other', '"': 'Other', '@': 'Other',
        'B': 'Normal' # Bazı kaynaklar B'yi Normal kabul eder
    }

    for rec_name, rec_path in orijinal_records:
        try:
            # 1. WFDB Verilerini Oku
            record = wfdb.rdrecord(os.path.join(rec_path, rec_name))
            annotation = wfdb.rdann(os.path.join(rec_path, rec_name), 'atr')

            # MLII veya V5 sinyalini seç (MLII varsa onu tercih et)
            if 'MLII' in record.sig_name:
                signal_index = record.sig_name.index('MLII')
            elif 'V5' in record.sig_name:
                signal_index = record.sig_name.index('V5')
            else:
                print(f"Uyarı: {rec_name} kaydında MLII veya V5 kanalı bulunamadı. Atlanıyor.")
                continue

            ekg_signal_raw = record.p_signal[:, signal_index]

            # 2. Pan-Tompkins Sinyal İşleme
            ekg_mwi = pan_tompkins_processor(ekg_signal_raw)

            # 3. R-Tepe Tespiti (MWI Sinyalinde)
            r_peak_indices_mwi = detect_r_peaks(ekg_mwi)

            # Tespiti MLII sinyalindeki en yüksek değere kaydırma (Pan-Tompkins'in önerilen son adımı)
            r_peak_indices = []
            for mwi_peak_idx in r_peak_indices_mwi:
                # MWI tepe çevresinde MLII sinyalindeki maksimumu bul
                search_window = int(0.05 * FS) # ± 50ms arama
                start = max(0, mwi_peak_idx - search_window)
                end = min(len(ekg_signal_raw), mwi_peak_idx + search_window)

                window_raw = ekg_signal_raw[start:end]
                if len(window_raw) > 0:
                    peak_offset = np.argmax(window_raw)
                    r_peak_indices.append(start + peak_offset)
                else:
                    r_peak_indices.append(mwi_peak_idx) # Geri dönüş olarak MWI indeksini kullan

            r_peak_indices = np.array(r_peak_indices)
            r_peak_times = r_peak_indices / FS

            # 4. RR Aralıklarını Hesaplama
            rr_post = np.diff(r_peak_times, prepend=0) # Sonraki RR
            rr_pre = np.diff(r_peak_times, append=r_peak_times[-1] + np.mean(rr_post[1:] if len(rr_post)>1 else [0.8])) # Önceki RR (Son tepe için ortalama ekle)
            rr_pre = np.roll(rr_pre, 1)[1:] # İlk elemanı at

            # 5. Yerel Ritim Ortalaması (Örn: Son 5 RR aralığı)
            rr_local_mean = np.zeros_like(rr_pre)
            for i in range(len(rr_pre)):
                # Kendisi dahil son 5 RR (pre) ortalaması
                mean_window = rr_pre[max(0, i-4):i+1]
                rr_local_mean[i] = np.mean(mean_window)

            # 6. Annotation Eşleştirme
            annotations_dict = dict(zip(annotation.sample, annotation.symbol))
            r_peak_annotations = []

            for r_idx in r_peak_indices:
                # R-tepesine en yakın WFDB annotation'ını bul
                closest_ann_idx = annotation.sample[np.argmin(np.abs(annotation.sample - r_idx))]

                # İndekslerin yakınlığını kontrol et (örneğin ±50ms içinde olmalı)
                if np.abs(closest_ann_idx - r_idx) <= int(0.05 * FS):
                    symbol = annotations_dict.get(closest_ann_idx, 'Q')
                else:
                    symbol = 'Q' # Eşleşme yoksa 'Unknown' olarak işaretle

                r_peak_annotations.append(symbol)


            # 7. Özellikleri Oluşturma
            for i, r_idx in enumerate(r_peak_indices):

                feature = {
                    'R_Peak_Index': r_idx,
                    'R_Peak_Time_s': r_peak_times[i],

                    # Morfoloji
                    'R_Peak_Amplitude_Raw': ekg_signal_raw[r_idx],
                    'R_Peak_Amplitude_MWI': ekg_mwi[r_idx],

                    # Zaman Alanı Metrikleri
                    'RR_Pre_s': rr_pre[i],
                    'RR_Post_s': rr_post[i],
                    'RR_Local_Mean_s': rr_local_mean[i],

                    # Sınıflandırma Etiketleri
                    'Annotation_Symbol': r_peak_annotations[i],
                    'Annotation_Category': annotation_map.get(r_peak_annotations[i], 'Unknown')
                }
                all_features.append(feature)

        except Exception as e:
            print(f"Hata oluştu: {rec_name} kaydı işlenirken: {e}")

    return pd.DataFrame(all_features)

# --- Yeni Veri Çerçevesini Oluşturma ---
new_feature_df = create_feature_df(orijinal_records, root_path)